In [1]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import time

In [2]:
def visit_url(url,seconds,browser):
    browser.visit(url)
    time.sleep(seconds)
    return browser.html

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

soup = bs(visit_url("https://mars.nasa.gov/news/",5,browser), 'html.parser')

results = soup.find('li', class_='slide')
news_title = results.find('h3').text
news_p = results.find('div',class_="article_teaser_body").text
print(f"Title: {news_title} \nDescription: {news_p}")

Title: NASA Wins 4 Webbys, 4 People's Voice Awards 
Description: Winners include the JPL-managed "Send Your Name to Mars" campaign, NASA's Global Climate Change website and Solar System Interactive.


In [4]:
#url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
#browser.visit(url)
#html = browser.html

soup = bs(visit_url("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars",0,browser), 'html.parser')
browser.click_link_by_id('full_image')

browser.click_link_by_partial_text('more info')

#html = browser.html
soup = bs(browser.html, 'html.parser')

image_url = soup.find("img", class_="main_image")
featured_image_url = browser.url.split("/spaceimages")[0] + image_url['src']
featured_image_url

C:\Users\nickv\Anaconda3\envs\PythonData2020\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19382_hires.jpg'

In [5]:
soup = bs(visit_url("https://twitter.com/marswxreport?lang=en",20,browser), 'html.parser')

articles = soup.find_all('article')

latest_report_found = False
i = 0
mars_weather = "No Report Found"
while latest_report_found == False and i < len(articles):
    spans = articles[i].find_all('span')
    if spans[4].text[0:11] == "InSight sol":
        latest_report_found = True
        mars_weather = spans[4].text
    i += 1
    
mars_weather

'InSight sol 525 (2020-05-18) low -93.6ºC (-136.5ºF) high -0.4ºC (31.3ºF)\nwinds from the WNW at 4.8 m/s (10.8 mph) gusting to 18.5 m/s (41.4 mph)\npressure at 7.10 hPa'

In [6]:
tables = pd.read_html("https://space-facts.com/mars/")

df = tables[0]
df.columns = ['Category', 'Value']

html_table = df.to_html(index=False)
print(html_table)

html_table = html_table.replace('\n', '')


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Category</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [7]:

soup = bs(visit_url("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars",0,browser), 'html.parser')
items = soup.find_all('div',class_='description')

#loop thru the links and add dictionary to list
hemisphere_image_urls = []
for item in items:
    link = item.find("a")
    title = link.text[0:len(link.text)-9]
    browser.click_link_by_partial_text(link.text)
    soup = bs(browser.html,'html.parser')
    image = browser.find_by_text('Sample')
    image_url = image['href']
    hemisphere_image_urls.append({"title": title, "img_url": image_url})
    browser.back()
hemisphere_image_urls

C:\Users\nickv\Anaconda3\envs\PythonData2020\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [8]:
browser.quit()